* Kennth's Part Cleaning up Data by Counties


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from pandas.tseries.offsets import MonthEnd
import re
pd.set_option('display.max_rows', None)

In [2]:
# import csv's
county_cases = pd.read_csv('Resources/county_cases.csv')
county_population = pd.read_csv('Resources/county_populations.csv')

In [3]:
# renaming columns
county_population = county_population.rename(columns={"CTYNAME": "County", "pop2020": "Population 2020"})

# removing 'county' from the values in County column
county_population['County'].replace(' County','',regex=True,inplace=True)
# county_population['County'] = county_population['County'].str.rstrip(' County')

# sorts counties alphabetically and resets the index
county_population = county_population.sort_values(by=['County'], ascending = True)
county_population = county_population.reset_index(drop=True)

# saving to csv
county_population.to_csv('Clean_County_Population.csv')

county_population.head()

,County,Population 2020,GrowthRate
0,Alameda,1684000,11.2991
1,Alpine,1061,-8.6133
2,Amador,41047,8.3492
3,Butte,235354,7.0058
4,Calaveras,45522,0.1188


In [4]:
# removing 'Out Of Country' and 'Unassigned' values from county series
clean_county_cases = county_cases[county_cases.county != 'Out Of Country']
clean_county_cases = clean_county_cases[clean_county_cases.county != 'Unassigned']
clean_county_cases.head()

,county,totalcountconfirmed,totalcountdeaths,newcountconfirmed,newcountdeaths,date
0,Santa Clara,151.0,6.0,151,6,3/18/2020
1,Santa Clara,183.0,8.0,32,2,3/19/2020
2,Santa Clara,246.0,8.0,63,0,3/20/2020
3,Santa Clara,269.0,10.0,23,2,3/21/2020
4,Santa Clara,284.0,13.0,15,3,3/22/2020


In [5]:
# renaming columns
clean_county_cases.columns = ['County', 'TotalCountConfirmed', 'TotalCountDeaths', 'NewCountConfirmed', 'NewCountDeaths', 'Date']

# changing Date series to datetime format
clean_county_cases['Date'] = pd.to_datetime(clean_county_cases['Date'])

# grouping df by county then date and retrieving max count values per month for each county
county_month_group = clean_county_cases.groupby(['County', clean_county_cases['Date'].dt.strftime('%B')]).max()

# renaming indexes
county_month_group.index.names = ['County', 'Month']

# reordering month index to  go by calendar order 
county_month_group = county_month_group.reindex(['March', 'April', 'May', 'June', 'July', 'August', 'September', 'October'], level=1)

# dropping NewCountConfirmed and NewCountDeaths series
county_month_group.drop(columns= ['NewCountConfirmed', 'NewCountDeaths'], inplace=True)

# saving to csv
county_month_group.to_csv("Total_Case_Count.csv")
county_month_group.head()

TotalCountConfirmed  TotalCountDeaths       Date
County  Month                                                  
Alameda March                269.0               7.0 2020-03-31
        April               1599.0              63.0 2020-04-30
        May                 3362.0             104.0 2020-05-31
        June                6067.0             135.0 2020-06-30
        July               11441.0             182.0 2020-07-31

*Alex's Part Cleaning up Hospital Data

In [6]:
#dependecis and setup
import pandas as pd
import numpy as np

#Output File
clean_date = "CleanCSV/ICU_county.csv"
#load file
county_data = "Resources/hospitals_by_county.csv"

icu_bed = pd.read_csv(county_data)
icu_bed_df = pd.DataFrame(icu_bed)
icu_bed_df.head()

,County,todays_date,hospitalized_covid_confirmed_patients,hospitalized_suspected_covid_patients,hospitalized_covid_patients,all_hospital_beds,icu_covid_confirmed_patients,icu_suspected_covid_patients,icu_available_beds
0,Humboldt,3/29/20,1.0,10.0,NaN,NaN,0.0,0.0,18.0
1,Lake,3/29/20,0.0,1.0,NaN,NaN,0.0,1.0,2.0
2,Kern,3/29/20,16.0,57.0,NaN,NaN,8.0,8.0,39.0
3,Yuba,3/29/20,2.0,12.0,NaN,NaN,2.0,4.0,2.0
4,Fresno,3/29/20,8.0,116.0,NaN,NaN,3.0,14.0,22.0


In [7]:
#Indexing the data by County
icu_bed_df.set_index("County", inplace=True)
icu_bed_df.head()

,todays_date,hospitalized_covid_confirmed_patients,hospitalized_suspected_covid_patients,hospitalized_covid_patients,all_hospital_beds,icu_covid_confirmed_patients,icu_suspected_covid_patients,icu_available_beds
County,,,,,,,,
Humboldt,3/29/20,1.0,10.0,NaN,NaN,0.0,0.0,18.0
Lake,3/29/20,0.0,1.0,NaN,NaN,0.0,1.0,2.0
Kern,3/29/20,16.0,57.0,NaN,NaN,8.0,8.0,39.0
Yuba,3/29/20,2.0,12.0,NaN,NaN,2.0,4.0,2.0
Fresno,3/29/20,8.0,116.0,NaN,NaN,3.0,14.0,22.0


In [8]:
#Using loc to find specefic counties and keeping the only columns we need
sorted_icu = icu_bed_df.loc[['Alameda', 'Amador', 'Butte', 'Calaveras', 'Colusa', 'Contra Costa', 'Del Norte',
                             'El Dorado', 'Fresno', 'Glenn','Humboldt', 'Imperial', 'Inyo', 'Kern', 'Kings', 'Lake', 
                             'Lassen', 'Los Angeles', 'Madera', 'Marin', 'Mariposa', 'Mendocino', 'Merced', 'Modoc',
                             'Mono', 'Monterey', 'Napa', 'Nevada', 'Orange', 'Placer',
                             'Plumas', 'Riverside', 'Sacramento', 'San Benito', 'San Bernardino', 'San Diego', 
                             'San Francisco', 'San Joaquin', 'San Luis Obispo','San Mateo', 'Santa Clara',
                             'Shasta','Sutter', 'Siskiyou', 'Solano', 'Sonoma','Santa Barbara', 
                             'Stanislaus', 'Tuolumne', 'Trinity', 'Tehama', 'Tulare','Ventura', 'Yolo', 'Yuba'],
                    ['todays_date', 'hospitalized_covid_confirmed_patients', 'hospitalized_suspected_covid_patients', 
                     'icu_covid_confirmed_patients', 'icu_suspected_covid_patients', 'icu_available_beds']]
sorted_icu.head()

,todays_date,hospitalized_covid_confirmed_patients,hospitalized_suspected_covid_patients,icu_covid_confirmed_patients,icu_suspected_covid_patients,icu_available_beds
County,,,,,,
Alameda,3/29/20,36.0,91.0,20.0,16.0,119.0
Alameda,3/30/20,59.0,89.0,27.0,18.0,146.0
Alameda,3/31/20,56.0,94.0,27.0,17.0,77.0
Alameda,4/1/20,52.0,76.0,27.0,15.0,106.0
Alameda,4/2/20,57.0,76.0,29.0,17.0,112.0


In [9]:
#Renaming the columns and clean up
icu_hospital_clean_data = sorted_icu.rename(columns={"hospitalized_covid_confirmed_patients": "Confirmed Covid Patients", 
                                         "hospitalized_suspected_covid_patients": "Suspected Covid Patinets",
                                        "todays_date": "Date", "icu_covid_confirmed_patients": "Confirmed Covid ICU Patients",
                                        "icu_suspected_covid_patients": "ICU suspected COVID Patients", "icu_available_beds": "ICU Beds Available", "county": "County"})
icu_hospital_clean_data.head()

,Date,Confirmed Covid Patients,Suspected Covid Patinets,Confirmed Covid ICU Patients,ICU suspected COVID Patients,ICU Beds Available
County,,,,,,
Alameda,3/29/20,36.0,91.0,20.0,16.0,119.0
Alameda,3/30/20,59.0,89.0,27.0,18.0,146.0
Alameda,3/31/20,56.0,94.0,27.0,17.0,77.0
Alameda,4/1/20,52.0,76.0,27.0,15.0,106.0
Alameda,4/2/20,57.0,76.0,29.0,17.0,112.0


In [10]:
# changing Date series to datetime format
icu_hospital_clean_data['Date'] = pd.to_datetime(icu_hospital_clean_data['Date'])

# grouping df by county then date and retrieving max count values per month for each county
final_icu_data = icu_hospital_clean_data.groupby(['County', icu_hospital_clean_data['Date'].dt.strftime('%B')]).max()
final_icu_data.head()

Date  Confirmed Covid Patients  Suspected Covid Patinets  \
County  Date                                                                    
Alameda April  2020-04-30                      93.0                     106.0   
        August 2020-08-31                     207.0                      56.0   
        July   2020-07-31                     213.0                      82.0   
        June   2020-06-30                     116.0                      92.0   
        March  2020-03-31                      59.0                      94.0   

                Confirmed Covid ICU Patients  ICU suspected COVID Patients  \
County  Date                                                                 
Alameda April                           46.0                          26.0   
        August                          73.0                          20.0   
        July                            65.0                          13.0   
        June                            43.0                          20.0   
        March                           27.0                          18.0   

                ICU Beds Available  
County  Date                        
Alameda April                197.0  
        August                99.0  
        July                 151.0  
        June                 203.0  
        March                146.0

In [11]:
#Exporting CSV to folder
final_icu_data.to_csv("CleanCSV/clean_hospitals_by_count.csv")

*Pheobe's Homelessness Data Cleaning

In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as sts
import numpy as np

In [13]:
data_file = "Resources/homeless_impact.csv"
homeless = pd.read_csv(data_file)
homeless.head()

,county,date,rooms,rooms_occupied,trailers_requested,trailers_delivered,donated_trailers_delivered
0,Alameda,4/14/20,403.0,115.0,91.0,85.0,NaN
1,Alpine,4/14/20,NaN,NaN,NaN,NaN,NaN
2,Amador,4/14/20,NaN,NaN,NaN,NaN,NaN
3,Butte,4/14/20,65.0,65.0,NaN,NaN,NaN
4,Calaveras,4/14/20,NaN,NaN,NaN,NaN,NaN


In [14]:
#remove columns that don't need
homeless_rooms = homeless.drop(['donated_trailers_delivered','trailers_requested','trailers_delivered'],axis=1)
homeless_rooms.head()

,county,date,rooms,rooms_occupied
0,Alameda,4/14/20,403.0,115.0
1,Alpine,4/14/20,NaN,NaN
2,Amador,4/14/20,NaN,NaN
3,Butte,4/14/20,65.0,65.0
4,Calaveras,4/14/20,NaN,NaN


In [15]:
#remove all NaN
homeless_rooms = homeless_rooms.dropna(how = 'any')
homeless_rooms.head()

,county,date,rooms,rooms_occupied
0,Alameda,4/14/20,403.0,115.0
3,Butte,4/14/20,65.0,65.0
9,Fresno,4/14/20,305.0,250.0
18,Los Angeles,4/14/20,2654.0,535.0
20,Marin,4/14/20,80.0,43.0


In [16]:
#Standardize county name
homeless_rooms=homeless_rooms.sort_values(by = ['county'],ascending=True)

homeless_rooms.county.unique()

array(['Alameda', 'Alameda ', 'Alameda County', 'Butte', 'Butte ',
       'Butte County', 'Calaveras', 'Calaveras ', 'Calaveras County',
       'Contra Costa', 'Contra Costa ', 'Contra Costa County',
       'Del Norte County', 'El Dorado County', 'Fresno', 'Fresno ',
       'Fresno County', 'Humboldt', 'Humboldt County', 'Imperial County',
       'Kern County', 'Kings County', 'Lassen County', 'Los Angeles',
       'Los Angeles ', 'Los Angeles County', 'Madera', 'Madera County ',
       'Marin', 'Marin ', 'Marin County ', 'Mariposa', 'Mariposa ',
       'Mariposa County', 'Mendocino', 'Mendocino ', 'Mendocino County',
       'Merced', 'Merced County', 'Monterey County', 'Napa', 'Napa ',
       'Napa County ', 'Nevada', 'Nevada County ', 'Orange', 'Orange ',
       'Orange County', 'Placer', 'Placer ', 'Placer County',
       'Plumas County', 'Riverside', 'Riverside ', 'Riverside County',
       'Sacramento', 'Sacramento ', 'Sacramento County', 'San Benito',
       'San Benito ', 'San B

In [17]:
for i in homeless_rooms.index:
    if (homeless_rooms.loc[i,'county']== 'Alameda ')| (homeless_rooms.loc[i,'county']== 'Alameda County'):
        homeless_rooms.loc[i,'county'] = 'Alameda'
    if (homeless_rooms.loc[i,'county']== 'Butte ')| (homeless_rooms.loc[i,'county']== 'Butte County'):
        homeless_rooms.loc[i,'county'] = 'Butte'
    if (homeless_rooms.loc[i,'county']== 'Calaveras ')| (homeless_rooms.loc[i,'county']== 'Calaveras County'):
        homeless_rooms.loc[i,'county'] = 'Calaveras'
    if (homeless_rooms.loc[i,'county']== 'Contra Costa ')| (homeless_rooms.loc[i,'county']== 'Contra Costa County'):
        homeless_rooms.loc[i,'county'] = 'Contra Costa'
    if (homeless_rooms.loc[i,'county']== 'Fresno ')| (homeless_rooms.loc[i,'county']== 'Fresno County'):
        homeless_rooms.loc[i,'county'] = 'Fresno'
    if (homeless_rooms.loc[i,'county']== 'Humboldt ')| (homeless_rooms.loc[i,'county']== 'Humboldt County'):
        homeless_rooms.loc[i,'county'] = 'Humboldt'
    if (homeless_rooms.loc[i,'county']== 'Los Angeles ')| (homeless_rooms.loc[i,'county']== 'Los Angeles County'):
        homeless_rooms.loc[i,'county'] = 'Los Angeles'
    if (homeless_rooms.loc[i,'county']== 'Madera ')| (homeless_rooms.loc[i,'county']== 'Madera County '):
        homeless_rooms.loc[i,'county'] = 'Madera'
    if (homeless_rooms.loc[i,'county']== 'Marin ')| (homeless_rooms.loc[i,'county']== 'Marin County '):
        homeless_rooms.loc[i,'county'] = 'Marin'
    if (homeless_rooms.loc[i,'county']== 'Del Norte')| (homeless_rooms.loc[i,'county']== 'Del Norte County'):
        homeless_rooms.loc[i,'county'] = 'Del Norte'
    if (homeless_rooms.loc[i,'county']== 'El Dorado ')| (homeless_rooms.loc[i,'county']== 'El Dorado County'):
        homeless_rooms.loc[i,'county'] = 'El Dorado'
    if (homeless_rooms.loc[i,'county']== 'Imperial ')| (homeless_rooms.loc[i,'county']== 'Imperial County'):
        homeless_rooms.loc[i,'county'] = 'Imperial'
    if (homeless_rooms.loc[i,'county']== 'Kern ')| (homeless_rooms.loc[i,'county']== 'Kern County'):
        homeless_rooms.loc[i,'county'] = 'Kern'
    if (homeless_rooms.loc[i,'county']== 'Kings ')| (homeless_rooms.loc[i,'county']== 'Kings County'):
        homeless_rooms.loc[i,'county'] = 'Kings'
    if (homeless_rooms.loc[i,'county']== 'Lassen ')| (homeless_rooms.loc[i,'county']== 'Lassen County'):
        homeless_rooms.loc[i,'county'] = 'Lassen'
    if (homeless_rooms.loc[i,'county']== 'Mariposa ')| (homeless_rooms.loc[i,'county']== 'Mariposa County'):
        homeless_rooms.loc[i,'county'] = 'Mariposa'
    if (homeless_rooms.loc[i,'county']== 'Mendocino ')| (homeless_rooms.loc[i,'county']== 'Mendocino County'):
        homeless_rooms.loc[i,'county'] = 'Mendocino'
    if (homeless_rooms.loc[i,'county']== 'Mariposa ')| (homeless_rooms.loc[i,'county']== 'Mariposa County'):
        homeless_rooms.loc[i,'county'] = 'Mariposa'
    if (homeless_rooms.loc[i,'county']== 'Merced ')| (homeless_rooms.loc[i,'county']== 'Merced County'):
        homeless_rooms.loc[i,'county'] = 'Merced'
    if (homeless_rooms.loc[i,'county']== 'Monterey ')| (homeless_rooms.loc[i,'county']== 'Monterey County'):
        homeless_rooms.loc[i,'county'] = 'Monterey'
    if (homeless_rooms.loc[i,'county']== 'Napa ')| (homeless_rooms.loc[i,'county']== 'Napa County '):
        homeless_rooms.loc[i,'county'] = 'Napa'
    if (homeless_rooms.loc[i,'county']== 'Nevada ')| (homeless_rooms.loc[i,'county']== 'Nevada County '):
        homeless_rooms.loc[i,'county'] = 'Nevada'
    if (homeless_rooms.loc[i,'county']== 'Orange ')| (homeless_rooms.loc[i,'county']== 'Orange County'):
        homeless_rooms.loc[i,'county'] = 'Orange'
    if (homeless_rooms.loc[i,'county']== 'Placer ')| (homeless_rooms.loc[i,'county']== 'Placer County'):
        homeless_rooms.loc[i,'county'] = 'Placer'
    if (homeless_rooms.loc[i,'county']== 'Plumas ')| (homeless_rooms.loc[i,'county']== 'Plumas County'):
        homeless_rooms.loc[i,'county'] = 'Plumas'
    if (homeless_rooms.loc[i,'county']== 'Riverside ')| (homeless_rooms.loc[i,'county']== 'Riverside County'):
        homeless_rooms.loc[i,'county'] = 'Riverside'
    if (homeless_rooms.loc[i,'county']== 'Sacramento ')| (homeless_rooms.loc[i,'county']== 'Sacramento County'):
        homeless_rooms.loc[i,'county'] = 'Sacramento'
    if (homeless_rooms.loc[i,'county']== 'San Benito ')| (homeless_rooms.loc[i,'county']== 'San Benito County'):
        homeless_rooms.loc[i,'county'] = 'San Benito'
    if (homeless_rooms.loc[i,'county']== 'San Bernardino ')| (homeless_rooms.loc[i,'county']== 'San Bernardino\xa0County'):
        homeless_rooms.loc[i,'county'] = 'San Bernardino'
    if (homeless_rooms.loc[i,'county']== 'San Diego ')| (homeless_rooms.loc[i,'county']== 'San Diego County'):
        homeless_rooms.loc[i,'county'] = 'San Diego'
    if (homeless_rooms.loc[i,'county']== 'San Joaquin ')| (homeless_rooms.loc[i,'county']== 'San Joaquin County'):
        homeless_rooms.loc[i,'county'] = 'San Joaquin'
    if (homeless_rooms.loc[i,'county']== 'San Mateo ')| (homeless_rooms.loc[i,'county']== 'San Mateo County'):
        homeless_rooms.loc[i,'county'] = 'San Mateo'
    if (homeless_rooms.loc[i,'county']== 'Santa Barbara ')| (homeless_rooms.loc[i,'county']== 'Santa Barbara County'):
        homeless_rooms.loc[i,'county'] = 'Santa Barbara'
    if (homeless_rooms.loc[i,'county']== 'Santa Clara ')| (homeless_rooms.loc[i,'county']== 'Santa Clara County'):
        homeless_rooms.loc[i,'county'] = 'Santa Clara'
    if (homeless_rooms.loc[i,'county']== 'Santa Cruz ')| (homeless_rooms.loc[i,'county']== 'Santa Cruz County'):
        homeless_rooms.loc[i,'county'] = 'Santa Cruz'
    if (homeless_rooms.loc[i,'county']== 'Shasta ')| (homeless_rooms.loc[i,'county']== 'Shasta County'):
        homeless_rooms.loc[i,'county'] = 'Shasta'
    if (homeless_rooms.loc[i,'county']== 'Siskiyou ')| (homeless_rooms.loc[i,'county']== 'Siskiyou County'):
        homeless_rooms.loc[i,'county'] = 'Siskiyou'
    if (homeless_rooms.loc[i,'county']== 'Solano ')| (homeless_rooms.loc[i,'county']== 'Solano County'):
        homeless_rooms.loc[i,'county'] = 'Solano'
    if (homeless_rooms.loc[i,'county']== 'Sonoma ')| (homeless_rooms.loc[i,'county']== 'Sonoma County'):
        homeless_rooms.loc[i,'county'] = 'Sonoma'
    if (homeless_rooms.loc[i,'county']== 'Stanislaus ')| (homeless_rooms.loc[i,'county']== 'Stanislaus County'):
        homeless_rooms.loc[i,'county'] = 'Stanislaus'
    if (homeless_rooms.loc[i,'county']== 'Sutter ')| (homeless_rooms.loc[i,'county']== 'Sutter County'):
        homeless_rooms.loc[i,'county'] = 'Sutter'
    if (homeless_rooms.loc[i,'county']== 'Tulare ')| (homeless_rooms.loc[i,'county']== 'Tulare County'):
        homeless_rooms.loc[i,'county'] = 'Tulare'
    if (homeless_rooms.loc[i,'county']== 'Tuolumne ')| (homeless_rooms.loc[i,'county']== 'Tuolumne County'):
        homeless_rooms.loc[i,'county'] = 'Tuolumne'
    if (homeless_rooms.loc[i,'county']== 'Ventura ')| (homeless_rooms.loc[i,'county']== 'Ventura County'):
        homeless_rooms.loc[i,'county'] = 'Ventura'
    if (homeless_rooms.loc[i,'county']== 'Yolo ')| (homeless_rooms.loc[i,'county']== 'Yolo County'):
        homeless_rooms.loc[i,'county'] = 'Yolo'
    if (homeless_rooms.loc[i,'county']== 'Yuba ')| (homeless_rooms.loc[i,'county']== 'Yuba County'):
        homeless_rooms.loc[i,'county'] = 'Yuba'

In [18]:
#list of counties
county_list_homeless=homeless_rooms["county"].drop_duplicates()
county_list_homeless= county_list_homeless.reset_index()
county_list_homeless.head()

,index,county
0,0,Alameda
1,293,Butte
2,352,Calaveras
3,586,Contra Costa
4,5740,Del Norte


In [19]:
homeless_rooms.head()

,county,date,rooms,rooms_occupied
0,Alameda,4/14/20,403.0,115.0
406,Alameda,4/21/20,403.0,346.0
464,Alameda,4/22/20,403.0,346.0
580,Alameda,4/24/20,385.0,344.0
290,Alameda,4/19/20,403.0,328.0


In [20]:
#Add Month colomn to organize data
homeless_rooms['date']=pd.to_datetime(homeless_rooms['date'])
homeless_rooms["Month"] = pd.DatetimeIndex(homeless_rooms["date"]).month
#homeless_rooms['Month'] = pd.to_datetime(homeless_rooms['Month'], format='%m').dt.month_name().str.slice(stop=10)
homeless_rooms.head()

,county,date,rooms,rooms_occupied,Month
0,Alameda,2020-04-14,403.0,115.0,4
406,Alameda,2020-04-21,403.0,346.0,4
464,Alameda,2020-04-22,403.0,346.0,4
580,Alameda,2020-04-24,385.0,344.0,4
290,Alameda,2020-04-19,403.0,328.0,4


In [21]:
#group data by Month
#homeless_rooms=homeless_rooms.set_index("Month")
homeless_rooms_month = homeless_rooms.groupby(['Month','county'],as_index=False).agg({'rooms': ['mean'],'rooms_occupied': ['mean']})
#homeless_rooms_month = homeless_rooms_month.reset_index()
#homeless_rooms_month.rename(columns={ homeless_rooms_month.columns[3]: "rooms_mean" }, inplace = True)
homeless_rooms_month.head()

,Month,county,rooms,rooms_occupied
,,,mean,mean
0,4,Alameda,395.588235,305.352941
1,4,Butte,86.058824,86.058824
2,4,Calaveras,48.533333,22.000000
3,4,Contra Costa,379.000000,165.625000
4,4,Fresno,305.000000,250.000000


In [22]:
homeless_rooms_month.to_csv('CleanCSV/Homeless.csv',index = False,header=False)

*Kevin's PPE Data Cleaning

In [29]:
# Import dependencies
import pandas as pd

import numpy as np
import matplotlib as plt

In [30]:
# Store large csv file into dataframe
filepath = 'Resources/logistics_ppe.csv'
ppe_data = pd.read_csv(filepath)
ppe_data.head()

,county,product_family,quantity_filled,shipping_zip_postal_code,as_of_date
0,Tulare,Other / None of the above,NaN,93257,2020-07-16
1,Tulare,Other / None of the above,NaN,93257,2020-07-16
2,Tulare,Other / None of the above,NaN,93257,2020-07-16
3,Tulare,Other / None of the above,NaN,93257,2020-07-16
4,Tulare,Other / None of the above,NaN,93257,2020-07-16


In [31]:
# Rename data columns
ppedf = pd.DataFrame()

ppedf = ppe_data.rename(columns = {'county':'County', 
                                      'product_family':'Product',
                                     'quantity_filled': 'Amount Fulfilled',
                                     'shipping_zip_postal_code': 'Postal',
                                     'as_of_date': 'Date'
                                     })
ppedf.head()

,County,Product,Amount Fulfilled,Postal,Date
0,Tulare,Other / None of the above,NaN,93257,2020-07-16
1,Tulare,Other / None of the above,NaN,93257,2020-07-16
2,Tulare,Other / None of the above,NaN,93257,2020-07-16
3,Tulare,Other / None of the above,NaN,93257,2020-07-16
4,Tulare,Other / None of the above,NaN,93257,2020-07-16


In [32]:
# Drop Unnecessary rows/columns

# Drop Postal column
ppedf.drop(['Postal'], axis = 1, inplace = True)

# Drop NaN rows
ppedf = ppedf.dropna()

# Drop rows with 0 fulfilled equipment
ppedf = ppedf[ppedf['Amount Fulfilled'] > 0]
ppedf.head()

,County,Product,Amount Fulfilled,Date
21,Orange,Viral Testing Media,2000.0,2020-07-16
22,Orange,Swabs,1000.0,2020-07-16
23,Butte,Surgical or Examination Gowns,1320.0,2020-07-16
24,Solano,Swabs,100.0,2020-07-16
25,Fresno,Swabs,200.0,2020-07-16


In [33]:
# Change Date column to be by month

# Change Date to datetime format
ppedf['Date'] = pd.to_datetime(ppedf['Date'])

# Extract Month String from datetime object
ppedf['Date'] = ppedf['Date'].dt.strftime('%B')
ppedf.head()

,County,Product,Amount Fulfilled,Date
21,Orange,Viral Testing Media,2000.0,July
22,Orange,Swabs,1000.0,July
23,Butte,Surgical or Examination Gowns,1320.0,July
24,Solano,Swabs,100.0,July
25,Fresno,Swabs,200.0,July


In [34]:
# use groupby, combine rows that have the same County, Product, and Month, and Sum the amount fulfilled

# agg_func = {'Amount Fulfilled':'sum'}
# ppedf.groupby(['County', 'Product', 'Date'])['Amount Fulfilled'].sum().to_frame().reset_index()
# ppedf

In [35]:
# Sort the values alphabetically by County and Date
ppedf = ppedf.sort_values(['County', 'Date'])
ppedf.head()

,County,Product,Amount Fulfilled,Date
204884,Alameda,N-95 Respirators,1260.0,August
204885,Alameda,Surgical Masks,2000.0,August
204931,Alameda,Surgical Masks,46000.0,August
204932,Alameda,Cloth Masks,50500.0,August
204933,Alameda,Face Shields (Disposable),10152.0,August


In [36]:
# Check value counts of County column to see which counties to exclude
ppedf['County'].value_counts()

State Agency               411873
Non-Governmental Entity    384686
Los Angeles                254565
State Agency or Other      108093
Sacramento                  79398
Orange                      72107
San Diego                   64273
Alameda                     62171
Riverside                   50821
San Joaquin                 50087
San Bernardino              49252
Monterey                    46859
Fresno                      43037
Tulare                      42465
Santa Clara                 36011
Imperial                    35041
Stanislaus                  34130
San Francisco               32588
Contra Costa                32199
Sonoma                      31184
San Mateo                   30318
Ventura                     29406
Kern                        29367
Yolo                        29253
Santa Cruz                  29048
Santa Barbara               28665
Tribal                      27548
Placer                      26683
Humboldt                    26577
Del Norte     

In [37]:
# Drop data that does not belong to any county
countyToDrop = ppedf[(ppedf['County'] == 'Governmental Entity') | 
                     (ppedf['County'] == 'State Agency') |
                    (ppedf['County'] == 'Non-Governmental Entity') |
                    (ppedf['County'] == 'State Agency or Other') |
                    (ppedf['County'] == 'Other')
                    
                    ].index
ppedf.drop(countyToDrop, inplace = True)
ppedf['County'].value_counts()

Los Angeles        254565
Sacramento          79398
Orange              72107
San Diego           64273
Alameda             62171
Riverside           50821
San Joaquin         50087
San Bernardino      49252
Monterey            46859
Fresno              43037
Tulare              42465
Santa Clara         36011
Imperial            35041
Stanislaus          34130
San Francisco       32588
Contra Costa        32199
Sonoma              31184
San Mateo           30318
Ventura             29406
Kern                29367
Yolo                29253
Santa Cruz          29048
Santa Barbara       28665
Tribal              27548
Placer              26683
Humboldt            26577
Del Norte           25580
Napa                25287
Solano              23900
San Luis Obispo     22684
Lake                22675
Merced              21273
Butte               20832
Marin               20321
Inyo                18351
Kings               17283
Tuolumne            17119
Mono                15511
Mendocino   

In [38]:
# Reset the index
ppedf = ppedf.reset_index()
ppedf.head()

,index,County,Product,Amount Fulfilled,Date
0,204884,Alameda,N-95 Respirators,1260.0,August
1,204885,Alameda,Surgical Masks,2000.0,August
2,204931,Alameda,Surgical Masks,46000.0,August
3,204932,Alameda,Cloth Masks,50500.0,August
4,204933,Alameda,Face Shields (Disposable),10152.0,August


In [39]:
# Output the final cleaned file to its csv
ppedf.to_csv('CleanCSV/PPE_data_clean.csv')